# Compute EKE using modeled data for comparison with observation 
The observational period for satellite altimetry stretches from 1993-mid 2020 and ssh anomalies used to calculate geostrophic velocity anomalies are relative to the reference period 1993-2012. Surface velocity anomalies within the merged modeled dataset will first be calculated using this reference period and EKE will be calculated using the velocity anomalies.

In [ ]:
import xarray as xr
import numpy as np
from cdo import *
cdo=Cdo()
from joblib import Parallel, delayed

Cut the datasets down to the necessary years. All of 2020 will be included in the 1993-2020 dataset, but only months of 2020 available in the observational dataset should be used in the final comparison.

In [ ]:
for i in [1,2,3,4,5]:
    inp='c6_ssh-vo_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol.nc'
    outp='c6_ssh-vo_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol.nc'
    cdo.selyear('1993/2020',input=inp,output=outp)
    
    inp='c6_ssh-uo_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol.nc'
    outp='c6_ssh-uo_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol.nc'
    cdo.selyear('1993/2020',input=inp,output=outp)

In [ ]:
for i in [1,2,3,4,5]:
    inp='c6_ssh-vo_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol.nc'
    outp='c6_ssh-vo_mon_lin_eq_5day_node_r'+str(i)+'_1993-2012_nol.nc'
    cdo.selyear('1993/2012',input=inp,output=outp)
    
    inp='c6_ssh-uo_mon_lin_eq_5day_node_r'+str(i)+'_1860-2090_nol.nc'
    outp='c6_ssh-uo_mon_lin_eq_5day_node_r'+str(i)+'_1993-2012_nol.nc'
    cdo.selyear('1993/2012',input=inp,output=outp)

Calculate anomalies based on the 1993-2012 mean of each 5-day period. 

In [ ]:
inds=[]
for j in np.arange(len(temp.time))[:73]:
    inds.append(np.arange((j),(j+(20*73)),73))

In [ ]:
def ref_mean(i,data):
    return np.mean(data[inds[i],:].values,axis=0) 

In [ ]:
#vo
for run in [1,2,3,4,5]:
    temp=xr.open_dataset('c6_ssh-vo_mon_lin_eq_5day_node_r'+str(run)+'_1993-2020_nol.nc')
    dtemp=temp.vo
    
    rmvals=Parallel(n_jobs=-1,batch_size='auto',verbose=10)(delayed(ref_mean)(i,dtemp) for i in np.arange(len(inds)))
    rmvals=np.asarray(rmvals)
    
    repeat=[]
    for i in np.arange(28):
        repeat.append(rmvals)
    rv=np.vstack(repeat)
    
    tempds=xr.DataArray(data=rv,coords={'time':temp.time[:]},
                          dims=['time','ncells'],
                          attrs={'description':'multi-year mean of five day vo values 1993-2012',
                                'units':'m/s',
                                })
    tempda=tempds.to_dataset(name='vo')
    tempda=tempda.assign({'time_bnds':temp.time_bnds[:]})
    tempda.to_netcdf('c6_ssh-vo_mon_lin_eq_5day_node_r'+str(run)+'_1993-2012_means.nc') 

In [ ]:
temp=xr.open_dataset('c6_ssh-vo_mon_lin_eq_5day_node_r'+str(1)+'_1993-2020_nol.nc')
temp

In [ ]:
#uo
for run in [1,2,3,4,5]:
    temp=xr.open_dataset('c6_ssh-uo_mon_lin_eq_5day_node_r'+str(run)+'_1993-2020_nol.nc')
    dtemp=temp.uo
    
    rmvals=Parallel(n_jobs=-1,batch_size='auto',verbose=10)(delayed(ref_mean)(i,dtemp) for i in np.arange(len(inds)))
    rmvals=np.asarray(rmvals)
    
    repeat=[]
    for i in np.arange(28):
        repeat.append(rmvals)
    rv=np.vstack(repeat)
    
    tempds=xr.DataArray(data=rv,coords={'time':temp.time[:]},
                          dims=['time','ncells'],
                          attrs={'description':'multi-year mean of five day uo values 1993-2012',
                                'units':'m/s',
                                })
    tempda=tempds.to_dataset(name='uo')
    tempda=tempda.assign({'time_bnds':temp.time_bnds[:]})
    tempda.to_netcdf('c6_ssh-uo_mon_lin_eq_5day_node_r'+str(run)+'_1993-2012_means.nc') 

In [ ]:
#subtract the refernce period mean from the observational period data
for i in [1,2,3,4,5]:
    inp1='c6_ssh-vo_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol.nc'
    inp2='c6_ssh-vo_mon_lin_eq_5day_node_r'+str(i)+'_1993-2012_means.nc'
    outp='c6_ssh-vo_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol_obsanom.nc'
    cdo.sub(input=([inp1,inp2]),output=outp)
    
    

In [ ]:
for i in [1,2,3,4,5]:
    inp1='c6_ssh-uo_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol.nc'
    inp2='c6_ssh-uo_mon_lin_eq_5day_node_r'+str(i)+'_1993-2012_means.nc'
    outp='c6_ssh-uo_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol_obsanom.nc'
    cdo.sub(input=([inp1,inp2]),output=outp)

Calculate EKE from the anomaly data

In [ ]:
#square the anomalies
for i in [1,2,3,4,5]:
    inp='c6_ssh-vo_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol_obsanom.nc'
    outp='c6_ssh-vo2_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol_obsanom.nc'
    cdo.mul(input=([inp,inp]),output=outp)

In [ ]:
for i in [1,2,3,4,5]:
    inp='c6_ssh-uo_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol_obsanom.nc'
    outp='c6_ssh-uo2_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol_obsanom.nc'
    cdo.mul(input=([inp,inp]),output=outp)

In [ ]:
#add the zonal and meridional squared velocities
for i in [1,2,3,4,5]:
    inpv='c6_ssh-vo2_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol_obsanom.nc'
    inpu='c6_ssh-uo2_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol_obsanom.nc'
    outp='c6_ssh-vo2-uo2_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol_obsanom.nc'
    
    cdo.add(input=([inpv,inpu]),output=outp)

In [ ]:
#multiply by the constant 1/2
for i in [1,2,3,4,5]:
    inp='c6_ssh-vo2-uo2_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol_obsanom.nc'
    outp='c6_ssh-eke_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol_obsanom.nc'
    cdo.mulc(0.5,input=inp,output=outp)

In [ ]:
#calculate the ensemble mean 
files=[]
for i in [1,2,3,4,5]:
    files.append('c6_ssh-eke_mon_lin_eq_5day_node_r'+str(i)+'_1993-2020_nol_obsanom.nc')
outp='c6_ssh-eke_mon_lin_eq_5day_node_ensmean_1993-2020_nol_obsanom.nc'
cdo.ensmean(input=(files),output=outp)

In [ ]:
#change the metadata to reflect the cdo operations
temp=xr.open_dataset('c6_ssh-eke_mon_lin_eq_5day_node_ensmean_1993-2020_nol_obsanom.nc')
temp=temp.rename({'vo':'EKE'})
temp['EKE']=temp.EKE.assign_attrs({'units':'m^2/s^2',
                                   'description':'Eddy kinetic energy computed using modeled 5day mean geostrophic surface velocity data. Anomalies based on 20yr reference period (1993-2012).',
                                   'data':'historic-ssp370 simulations from AWICM1s CMIP6 contribution. ensemble mean of five ensemble members'})

temp.to_netcdf('c6_ssh-eke_mon_lin_eq_5day_node_ensmean_1993-2020_nol_obsanom_mdata.nc')